In [46]:
import os
import json
import pandas as pd
import traceback

In [47]:
from langchain.chat_models import ChatOpenAI

In [48]:
from dotenv import load_dotenv
load_dotenv() # take the environment variable from .env

True

In [49]:
KEY=os.getenv("OPENAI_API_KEY")

In [50]:
llm= ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [51]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [52]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    }
}


In [53]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and user it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [54]:
quiz_genration_prompt =  PromptTemplate(input_variables=["text","number","subject","tone","response_json"], template=TEMPLATE)

In [55]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_genration_prompt, output_key="quiz", verbose=True)

In [56]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Mutiple choice Quiz for {subject} students. \
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity.
if the quiz is not at per with the cognitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that is perfectly fits the students ability.
Quiz_MCQa:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [57]:
quiz_evaluation_prompt =  PromptTemplate(input_variables=["subject","quiz"], template=TEMPLATE2)

In [58]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [59]:
generate_evaluate_chain  = SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],output_variables=["subject","quiz"],)

In [60]:
file_path=r"/Users/nk/WorkingDirectory/Python/mcqgen/data.txt"

In [61]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [62]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [63]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"

In [64]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other

In [65]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"CompletionTokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1803
Prompt Tokens:1375
CompletionTokens:428
Total Cost:0.0029185


In [67]:
quiz=response.get("quiz")

In [69]:
quiz=json.loads(quiz)

In [70]:
quiz_table_date = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join([
        f"{options}:{option_value}"
        for options, option_value in value["options"].items()
    ])
    correct = value["correct"]
    quiz_table_date.append({"MCQ": mcq, "Choices":options, "Correct":correct})

In [71]:
quiz_table_date

[{'MCQ': 'What term was coined in 1959 by Arthur Samuel to describe the field of computer gaming and artificial intelligence?',
  'Choices': 'a:Neural Networks | b:Machine Learning | c:Artificial Intelligence | d:Deep Learning',
  'Correct': 'b'},
 {'MCQ': 'Who introduced a theoretical neural structure in 1949 that set a groundwork for how AIs and machine learning algorithms work?',
  'Choices': 'a:Raytheon Company | b:Donald Hebb | c:Walter Pitts | d:Warren McCulloch',
  'Correct': 'b'},
 {'MCQ': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'Choices': 'a:Cyberdyne | b:Cybertron | c:Skynet | d:Matrix',
  'Correct': 'b'},
 {'MCQ': 'According to Tom M. Mitchell, what is the definition of a computer program learning from experience?',
  'Choices': 'a:Improving performance at tasks in T | b:Learning to recognize patterns | c:Classifying data based on models | d:Making predictions for future outcomes',
  'Correct': 'a'},
 {'

In [75]:
quiz=pd.DataFrame(quiz_table_date)

In [76]:
quiz.to_csv("machinelerning.csv")